In [2]:
import requests
import pandas
from urllib.parse import quote
import json
import re

ELEV 高度，單位 公尺
WDIR 風向，單位 度，一般風向 0 表示無風
WDSD 風速，單位 公尺/秒
TEMP 溫度，單位 攝氏
HUMD 相對濕度，單位 百分比率，此處以實數 0-1.0 記錄
PRES 測站氣壓，單位 百帕
24R 日累積雨量，單位 毫米
H_FX 小時最大陣風風速，單位 公尺/秒
H_XD 小時最大陣風風向，單位 度
H_FXT 小時最大陣風時間，hhmm (小時分鐘)
H_F10 本時最大10分鐘平均風速，單位 公尺/秒
H_10D 本時最大10分鐘平均風向，單位 度
H_F10T 本時最大10分鐘平均風速發生時間，hhmm (小時分鐘)
H_UVI 小時紫外線指數
D_TX 本日最高溫，單位 攝氏
D_TXT 本日最高溫發生時間，hhmm (小時分鐘)
D_TN 本日最低溫，單位 攝氏
D_TNT 本日最低溫發生時間，hhmm (小時分鐘)
D_TS 本日總日照時數，單位 小時
VIS 十分鐘盛行能見度，單位 公里
Weather 十分鐘天氣現象描述

10分鐘 api更新一次(ex 2:40分時 最後更新時間為2:30)

api = https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0003-001?Authorization=CWB-0BFC5710-D0E8-46A4-9B4A-6386C222F445&stationId={測站ID}


In [3]:
def request_api(stationid,url):
    #request_api
    weather_res = requests.get(url)
    weather_json = json.loads(weather_res.text)
    return weather_json

In [4]:

#main code
stationid = input("輸入測站ID: ")    #ex: stationid = "467440"
test = re.search(r'\d+',stationid).group(0)
weather_element = []
url = ""


weather_element1=["高度(m)","風向(度)","風速(m/s)","溫度(°C)","相對濕度(%)","測站氣壓(百帕)","日累積雨量(mm)","小時最大陣風風速(m/s)",
                 "小時最大陣風風向(度)","小時最大陣風時間(小時分鐘)","本時最大10分鐘平均風速(m/s)","本時最大10分鐘平均風向(度)",
                 "本時最大10分鐘平均風速發生時間(小時分鐘)","小時紫外線指數","本日最高溫(°C)","本日最高溫發生時間(小時分鐘)",
                 "本日最低溫(°C)","本日最低溫發生時間(小時分鐘)","本日總日照時數(hr)","十分鐘盛行能見度(km)","十分鐘天氣現象描述"]

weather_element2=["高度(m)","風向(度)","風速(m/s)","溫度(°C)","相對濕度(%)","測站氣壓(百帕)","日累積雨量(mm)","小時最大陣風風速(m/s)",
                 "小時最大陣風風向(度)","小時最大陣風時間(yyyy-MM-ddThh:mm:ss+08:00)","本日最高溫(°C)","本日最高溫發生時間(小時分鐘)",
                 "本日最低溫(°C)","本日最低溫發生時間(小時分鐘)"]
url1 = f"https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0003-001?Authorization=CWB-0BFC5710-D0E8-46A4-9B4A-6386C222F445&stationId={stationid}"
url2 = f"https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0001-001?Authorization=CWB-0BFC5710-D0E8-46A4-9B4A-6386C222F445&stationId={stationid}"


if stationid == test:
    url = url1
    weather_element = weather_element1
else:
    url = url2
    weather_element = weather_element2
    

weather_json = request_api(stationid,url)


#station data
list_station=["測站","測站ID","經度","緯度","最後更新時間"]
station_data=[]

station_data.append(weather_json["records"]["location"][0]["locationName"])
station_data.append(weather_json["records"]["location"][0]["stationId"])
station_data.append(weather_json["records"]["location"][0]["lat"])
station_data.append(weather_json["records"]["location"][0]["lon"])
station_data.append(weather_json["records"]["location"][0]["time"]["obsTime"])

station_dict = dict(zip( list_station , station_data ))
station_df = pandas.DataFrame( list( station_dict.items() ) , columns=["name","value"] )                                
station_df.index+=1

station_df

輸入測站ID: 467440


,name,value
1,測站,高雄
2,測站ID,467440
3,經度,22.567733
4,緯度,120.307664
5,最後更新時間,2022-05-31 09:40:00


In [6]:

#setting weather data
value=[]
weather_dict = {}


#get weather data value
for i in weather_json["records"]["location"][0]["weatherElement"]:
    value.append(i["elementValue"])
    

#input data to dict
weather_dict = dict(zip(weather_element,value))

#turn dict to dataframe
weather_df = pandas.DataFrame( list( weather_dict.items() ) , columns=['name','value(-99 == no data)'])
weather_df.index += 1
weather_df

,name,value(-99 == no data)
1,高度(m),2.30
2,風向(度),170
3,風速(m/s),2.10
4,溫度(°C),30.80
5,相對濕度(%),0.70
6,測站氣壓(百帕),1008.70
7,日累積雨量(mm),0
8,小時最大陣風風速(m/s),5.90
9,小時最大陣風風向(度),140
10,小時最大陣風時間(小時分鐘),0839
